In [1]:
#Imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [2]:
# load data
import constants
df = pd.read_csv("kddcup.data_10_percent.gz", names = constants.names)

#https://github.com/chinskiy/KDD-99
# Categorical features to numeric labels

from sklearn import preprocessing
le_dicts = {}

for categorical_name in constants.categorical_names:
    le = preprocessing.LabelEncoder()
    le.fit(df[categorical_name])
    le_dicts[categorical_name] = dict(zip(le.transform(le.classes_), le.classes_))
    df[categorical_name + '_num'] = le.fit_transform(df[categorical_name])

df['cible'] = df.label.apply(lambda label: 1 if label == 'normal.' else 0)
df['label_four'] = df.label.apply(lambda label: constants.label_to_four_attack_class[label])
df['label_four_num'] = df.label_four.apply(lambda label: constants.five_classes_to_num[label])
df.drop(constants.categorical_names + ['label','label_four_num','label_four'], axis=1, inplace=True)
df.shape

(494021, 42)

In [3]:
# first 5 
df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,protocol_type_num,service_num,flag_num,cible
0,0,181,5450,0,0,0,0,0,1,0,...,0.11,0.0,0.0,0.0,0.0,0.0,1,22,9,1
1,0,239,486,0,0,0,0,0,1,0,...,0.05,0.0,0.0,0.0,0.0,0.0,1,22,9,1
2,0,235,1337,0,0,0,0,0,1,0,...,0.03,0.0,0.0,0.0,0.0,0.0,1,22,9,1
3,0,219,1337,0,0,0,0,0,1,0,...,0.03,0.0,0.0,0.0,0.0,0.0,1,22,9,1
4,0,217,2032,0,0,0,0,0,1,0,...,0.02,0.0,0.0,0.0,0.0,0.0,1,22,9,1


In [4]:
# df to values
df = df.values

In [5]:
# GC Forest
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/kdd.json")  # layer = 1   k=10   Extree + DTree
gc = GCForest(config)

In [7]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,41]
X = df[:,0:41]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
    # X_enc is the concatenated predict_proba result of each estimators of the last layer of the GCForest model
    # X_enc.shape =
    #   (n_datas, n_estimators * n_classes): If cascade is provided
    #   (n_datas, n_estimators * n_classes, dimX, dimY): If only finegrained part is provided
    # You can also pass X_test, y_test to fit_transform method, then the accracy on test data will be logged when training.
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    # WARNING: if you set gc.set_keep_model_in_mem(True), you would have to use
    # gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test) to evaluate your model. 

[ 2018-04-21 14:32:17,495][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 41)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 41)],y_test.shape=(163027,)
[ 2018-04-21 14:32:17,572][cascade_classifier.fit_transform] group_dims=[41]
[ 2018-04-21 14:32:17,574][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-21 14:32:17,575][cascade_classifier.fit_transform] group_ends=[41]
[ 2018-04-21 14:32:17,577][cascade_classifier.fit_transform] X_train.shape=(330994, 41),X_test.shape=(163027, 41)
[ 2018-04-21 14:32:17,655][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 41), X_cur_test.shape=(163027, 41)
[ 2018-04-21 14:32:20,061][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.98%
[ 2018-04-21 14:32:22,262][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.97%
[ 2018-04-21 14:32:28,562][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

[ 2018-04-21 14:36:03,925][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_1 - 10_folds.train_8.predict)=99.98%
[ 2018-04-21 14:36:06,057][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_1 - 10_folds.train_9.predict)=99.99%
[ 2018-04-21 14:36:06,326][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_1 - 10_folds.train_cv.predict)=99.98%
[ 2018-04-21 14:36:06,331][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_1 - 10_folds.test.predict)=99.98%
[ 2018-04-21 14:36:27,322][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_2 - 10_folds.train_0.predict)=99.14%
[ 2018-04-21 14:36:49,792][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_2 - 10_folds.train_1.predict)=99.67%
[ 2018-04-21 14:37:10,257][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_2 - 10_folds.train_2.predict)=98.68%
[ 2018-04-21 14:37:35,296][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_2 - 10_folds.train_3.predict)=99.77%
[ 

[ 2018-04-21 14:43:43,477][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_0 - 10_folds.train_8.predict)=99.99%
[ 2018-04-21 14:43:45,712][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_0 - 10_folds.train_9.predict)=99.98%
[ 2018-04-21 14:43:46,004][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_0 - 10_folds.train_cv.predict)=99.98%
[ 2018-04-21 14:43:46,010][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_0 - 10_folds.test.predict)=99.98%
[ 2018-04-21 14:43:48,238][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 10_folds.train_0.predict)=99.98%
[ 2018-04-21 14:43:50,742][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 10_folds.train_1.predict)=99.99%
[ 2018-04-21 14:43:53,012][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 10_folds.train_2.predict)=99.99%
[ 2018-04-21 14:43:55,343][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 10_folds.train_3.predict)=99.98%
[ 

[ 2018-04-21 14:51:18,491][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_2 - 10_folds.test.predict)=99.01%
[ 2018-04-21 14:51:18,508][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=99.98%
[ 2018-04-21 14:51:18,512][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=99.98%
[ 2018-04-21 14:51:18,608][cascade_classifier.fit_transform] [layer=5] look_indexs=[0], X_cur_train.shape=(330994, 47), X_cur_test.shape=(163027, 47)
[ 2018-04-21 14:51:21,612][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 10_folds.train_0.predict)=99.99%
[ 2018-04-21 14:51:23,896][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 10_folds.train_1.predict)=99.99%
[ 2018-04-21 14:51:26,289][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 10_folds.train_2.predict)=99.99%
[ 2018-04-21 14:51:28,585][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 10_folds.train_3.predict)=99.98%
[ 2

[ 2018-04-21 14:55:59,661][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_1 - 10_folds.test.predict)=99.98%
[ 2018-04-21 14:56:21,560][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_2 - 10_folds.train_0.predict)=99.66%
[ 2018-04-21 14:56:43,493][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_2 - 10_folds.train_1.predict)=98.65%
[ 2018-04-21 14:57:04,410][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_2 - 10_folds.train_2.predict)=98.96%
[ 2018-04-21 14:57:28,693][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_2 - 10_folds.train_3.predict)=99.83%
[ 2018-04-21 14:57:52,716][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_2 - 10_folds.train_4.predict)=99.38%
[ 2018-04-21 14:58:15,933][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_2 - 10_folds.train_5.predict)=99.45%
[ 2018-04-21 14:58:32,646][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_2 - 10_folds.train_6.predict)=98.39%
[ 2

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of GCForest = {:.6f} %".format(acc * 100))

[ 2018-04-21 15:03:42,252][cascade_classifier.transform] X_groups_test.shape=[(163027, 41)]
[ 2018-04-21 15:03:42,283][cascade_classifier.transform] group_dims=[41]
[ 2018-04-21 15:03:42,285][cascade_classifier.transform] X_test.shape=(163027, 41)
[ 2018-04-21 15:03:42,312][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(163027, 41)
[ 2018-04-21 15:03:48,301][cascade_classifier.transform] [layer=1] look_indexs=[0], X_cur_test.shape=(163027, 47)
[ 2018-04-21 15:03:54,191][cascade_classifier.transform] [layer=2] look_indexs=[0], X_cur_test.shape=(163027, 47)
[ 2018-04-21 15:04:00,120][cascade_classifier.transform] [layer=3] look_indexs=[0], X_cur_test.shape=(163027, 47)
[ 2018-04-21 15:04:06,033][cascade_classifier.transform] [layer=4] look_indexs=[0], X_cur_test.shape=(163027, 47)


Test Accuracy of GCForest = 99.980985 %


# Neural network

In [10]:
# estimators 
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential # initialize neural network library
from keras.layers import Dense # build our layers library
def build_classifier():
    classifier = Sequential() # initialize neural network
    classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = X_train.shape[1]))
    classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu'))

    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
mean = accuracies.mean()
variance = accuracies.std()
print("Accuracy mean: "+ str(mean))
print("Accuracy variance: "+ str(variance))

Using TensorFlow backend.


Epoch 1/100
297894/297894 [==============================] - 20s - loss: 0.0326 - acc: 0.9875    
Epoch 2/100
297894/297894 [==============================] - 20s - loss: 0.0182 - acc: 0.9915    
Epoch 3/100
297894/297894 [==============================] - 20s - loss: 0.0172 - acc: 0.9931    
Epoch 4/100
297894/297894 [==============================] - 19s - loss: 0.0137 - acc: 0.9953    
Epoch 5/100
297894/297894 [==============================] - 19s - loss: 0.0154 - acc: 0.9966    
Epoch 6/100
297894/297894 [==============================] - 19s - loss: 0.0167 - acc: 0.9974    
Epoch 7/100
297894/297894 [==============================] - 19s - loss: 0.0097 - acc: 0.9979    
Epoch 8/100
297894/297894 [==============================] - 19s - loss: 0.0096 - acc: 0.9981    
Epoch 9/100
297894/297894 [==============================] - 19s - loss: 0.0089 - acc: 0.9982    
Epoch 10/100
297894/297894 [==============================] - 19s - loss: 0.0091 - acc: 0.9985    
Epoch 11/100
297894

297894/297894 [==============================] - 20s - loss: 0.0053 - acc: 0.9991    
Epoch 66/100
297894/297894 [==============================] - 19s - loss: 0.0052 - acc: 0.9992    
Epoch 67/100
297894/297894 [==============================] - 18s - loss: 0.0058 - acc: 0.9992    
Epoch 68/100
297894/297894 [==============================] - 19s - loss: 0.0062 - acc: 0.9992    
Epoch 69/100
297894/297894 [==============================] - 19s - loss: 0.0052 - acc: 0.9992    
Epoch 70/100
297894/297894 [==============================] - 19s - loss: 0.0055 - acc: 0.9989    
Epoch 71/100
297894/297894 [==============================] - 18s - loss: 0.0187 - acc: 0.9984    
Epoch 72/100
297894/297894 [==============================] - 18s - loss: 0.0047 - acc: 0.9992    
Epoch 73/100
297894/297894 [==============================] - 18s - loss: 0.0053 - acc: 0.9993    
Epoch 74/100
297894/297894 [==============================] - 19s - loss: 0.0053 - acc: 0.9992    
Epoch 75/100
297894/297

297894/297894 [==============================] - 20s - loss: 0.0099 - acc: 0.9987    
Epoch 29/100
297894/297894 [==============================] - 20s - loss: 0.0083 - acc: 0.9989    
Epoch 30/100
297894/297894 [==============================] - 20s - loss: 0.0067 - acc: 0.9990    
Epoch 31/100
297894/297894 [==============================] - 21s - loss: 0.0068 - acc: 0.9990    
Epoch 32/100
297894/297894 [==============================] - 19s - loss: 0.0067 - acc: 0.9990    
Epoch 33/100
297894/297894 [==============================] - 19s - loss: 0.0097 - acc: 0.9989    
Epoch 34/100
297894/297894 [==============================] - 19s - loss: 0.0064 - acc: 0.9991    
Epoch 35/100
297894/297894 [==============================] - 19s - loss: 0.0088 - acc: 0.9989    
Epoch 36/100
297894/297894 [==============================] - 19s - loss: 0.0093 - acc: 0.9989    
Epoch 37/100
297894/297894 [==============================] - 20s - loss: 0.0082 - acc: 0.9990    
Epoch 38/100
297894/297

297895/297895 [==============================] - 20s - loss: 0.0270 - acc: 0.9940    
Epoch 93/100
297895/297895 [==============================] - 20s - loss: 0.0270 - acc: 0.9940    
Epoch 94/100
297895/297895 [==============================] - 19s - loss: 0.0273 - acc: 0.9940    
Epoch 95/100
297895/297895 [==============================] - 19s - loss: 0.0277 - acc: 0.9940    
Epoch 96/100
297895/297895 [==============================] - 21s - loss: 0.0277 - acc: 0.9939    
Epoch 97/100
297895/297895 [==============================] - 19s - loss: 0.0269 - acc: 0.9940    
Epoch 98/100
297895/297895 [==============================] - 19s - loss: 0.0269 - acc: 0.9940    
Epoch 99/100
297895/297895 [==============================] - 19s - loss: 0.0502 - acc: 0.9940    
Epoch 100/100
297895/297895 [==============================] - 19s - loss: 0.0631 - acc: 0.9853    
Epoch 2/100
297895/297895 [==============================] - 17s - loss: 0.0419 - acc: 0.9895    
Epoch 3/100
297895/2978

297895/297895 [==============================] - 21s - loss: 0.0083 - acc: 0.9989    
Epoch 57/100
297895/297895 [==============================] - 20s - loss: 0.0470 - acc: 0.9965    
Epoch 58/100
297895/297895 [==============================] - 20s - loss: 0.0438 - acc: 0.9968    
Epoch 59/100
297895/297895 [==============================] - 20s - loss: 0.0106 - acc: 0.9989    
Epoch 60/100
297895/297895 [==============================] - 20s - loss: 0.0082 - acc: 0.9990    
Epoch 61/100
297895/297895 [==============================] - 23s - loss: 0.0105 - acc: 0.9989    
Epoch 62/100
297895/297895 [==============================] - 28s - loss: 0.0082 - acc: 0.9990    
Epoch 63/100
297895/297895 [==============================] - 22s - loss: 0.0084 - acc: 0.9991    
Epoch 64/100
297895/297895 [==============================] - 19s - loss: 0.0057 - acc: 0.9993    
Epoch 65/100
297895/297895 [==============================] - 20s - loss: 0.0064 - acc: 0.9992    
Epoch 66/100
297895/297

297895/297895 [==============================] - 21s - loss: 0.0778 - acc: 0.9945    
Epoch 20/100
297895/297895 [==============================] - 21s - loss: 0.0774 - acc: 0.9945    
Epoch 21/100
297895/297895 [==============================] - 20s - loss: 0.0775 - acc: 0.9945    
Epoch 22/100
297895/297895 [==============================] - 19s - loss: 0.0772 - acc: 0.9946    
Epoch 23/100
297895/297895 [==============================] - 20s - loss: 0.0785 - acc: 0.9946    
Epoch 24/100
297895/297895 [==============================] - ETA: 0s - loss: 0.0768 - acc: 0.994 - 20s - loss: 0.0769 - acc: 0.9947    
Epoch 25/100
297895/297895 [==============================] - 20s - loss: 0.0770 - acc: 0.9946    
Epoch 26/100
297895/297895 [==============================] - 20s - loss: 0.0772 - acc: 0.9946    
Epoch 27/100
297895/297895 [==============================] - 20s - loss: 0.0769 - acc: 0.9947    
Epoch 28/100
297895/297895 [==============================] - 20s - loss: 0.0771 - a

297895/297895 [==============================] - 21s - loss: 0.0062 - acc: 0.9990    
Epoch 84/100
297895/297895 [==============================] - 21s - loss: 0.0056 - acc: 0.9988    
Epoch 85/100
297895/297895 [==============================] - 21s - loss: 0.0052 - acc: 0.9990    
Epoch 86/100
297895/297895 [==============================] - 20s - loss: 0.0054 - acc: 0.9989    
Epoch 87/100
297895/297895 [==============================] - 21s - loss: 0.0057 - acc: 0.9986    
Epoch 88/100
297895/297895 [==============================] - 20s - loss: 0.0052 - acc: 0.9990    
Epoch 89/100
297895/297895 [==============================] - 21s - loss: 0.0053 - acc: 0.9990    
Epoch 90/100
297895/297895 [==============================] - 20s - loss: 0.0055 - acc: 0.9992    
Epoch 91/100
297895/297895 [==============================] - 20s - loss: 0.0063 - acc: 0.9990    
Epoch 92/100
297895/297895 [==============================] - 19s - loss: 0.0062 - acc: 0.9990    
Epoch 93/100
297895/297

# DecisionTreeClassifier

In [11]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Test Accuracy of DecisionTreeClassifier = {:.6f} %".format(acc * 100))

Test Accuracy of DecisionTreeClassifier = 99.968103 %


# RandomForestClassifier

In [12]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Test Accuracy of RandomForestClassifier = {:.6f} %".format(acc * 100))

Test Accuracy of RandomForestClassifier = 99.980985 %


# ExtraTreesClassifier

In [13]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Test Accuracy of ExtraTreesClassifier = {:.6f} %".format(acc * 100))

Test Accuracy of ExtraTreesClassifier = 99.974851 %


# AdaBoostClassifier

In [14]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Test Accuracy of AdaBoostClassifier = {:.6f} %".format(acc * 100))

Test Accuracy of AdaBoostClassifier = 99.929460 %


# Naive_bayes  GaussianNB

In [16]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb = gnb.fit(X_train, y_train)

y_pred= gnb.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Test Accuracy of GaussianNB = {:.6f} %".format(acc * 100))

Test Accuracy of GaussianNB = 91.964521 %
